# Setup
The following cells need to be run in order to use the Java kernel and set dependencies

In [ ]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

In [ ]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>org.slf4j</groupId>
    <artifactId>slf4j-simple</artifactId>
    <version>1.7.25</version>
  </dependency>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>6.1.4</version>
  </dependency>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-document-api</artifactId>
    <version>1.2.0</version>
  </dependency>
  <dependency>
    <groupId>commons-io</groupId>
    <artifactId>commons-io</artifactId>
    <version>2.6</version>
  </dependency>
  <dependency>
    <groupId>com.googlecode.json-simple</groupId>
    <artifactId>json-simple</artifactId>
    <version>1.1.1</version>
</dependency>
</dependencies>

## Import dependencies, Connect to Aerospike

In [ ]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.Bin;
import com.aerospike.client.Host;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.documentapi.*;
import com.aerospike.documentapi.JsonConverters;

import com.fasterxml.jackson.databind.JsonNode;
import com.jayway.jsonpath.JsonPath;
import java.util.List;
import java.util.Map;
import java.nio.charset.StandardCharsets;
import org.apache.commons.io.FileUtils;
import org.json.simple.JSONArray;
import org.json.simple.JSONObject;
import org.json.simple.parser.JSONParser;

Host[] hosts = new Host[] {
    new Host("127.0.0.1", 3000)
};

WritePolicy writePolicy = new WritePolicy();
writePolicy.sendKey = true;

AerospikeClient client = new AerospikeClient(null, hosts);
System.out.println("Initialized Aerospike client and connected to the cluster.");

AerospikeDocumentClient docClient = new AerospikeDocumentClient(client);
System.out.println("Initialized document client from the Aerospike client.");

# JSON Document

## Document Format

The document is an array of objects like the example below:

```json
{
    "_id": "58e3dc8ede9c102b036044e7",
    "company": {
        "_id": "58e3dcf2de9c102b036044e8",
        "about": "Solenix is a privately owned, independent and internationally active company...",
        "backgroundImage": "https://spaceindividuals.com/var/www/images/52251f825f0c59bf2ee429c075262fa8.jpg",
        "companyLocation": "Bornstrasse 3, 4616 Kappel, Switzerland",
        "companyName": "Solenix GmbH",
        "createdAt": "2018-03-03T14:00:04.000Z",
        "culture": "- Attractive salary, benefits and rewards. Besides the standard benefits you would expect...",
        "email": "career@solenix.ch",
        "galleryImages": [
            "https://spaceindividuals.com/var/www/images/be4dc86c6921c9f0729241f5f81dbaec.jpg",
            "https://spaceindividuals.com/var/www/images/3e13ef7d2b8cb7b6b059994e68398d9c.jpg",
            "https://spaceindividuals.com/var/www/images/0bf399d70514c1edf6d913e4edbabd4f.jpg"
        ],
        "isPublished": true,
        "kindOfEmployeeLookFor": null,
        "logoURL": "https://spaceindividuals.com/var/www/images/3b71131a4a213e1f17af26bf80ac0d97.png",
        "mission": null,
        "paymentType": "free",
        "phone": "+41 62 216 35 02",
        "sizeCompany": "25-50 employees",
        "specialities": "Solenix provides consultancy and software engineering services...",
        "testimonials": [],
        "weOfferChallenges": null,
        "weOfferKindOfChallenges": "Whether you are passionate about the space industry or you want...",
        "weOfferKindOfTrainings": null,
        "weOfferTrainings": null,
        "webSite": "http://www.solenix.ch"
    },
    "createdAt": "2018-03-03T14:00:04.000Z",
    "email": "mara.mcclain.pleskot@solenix.ch",
    "jobs": null,
    "numberOfJobs": 0,
    "updatedAt": "2020-11-09T14:27:50.753Z"
}
```

## Parse the JSON file

In [ ]:
String companiesString = FileUtils.readFileToString(new File("./space-companies.json"), StandardCharsets.UTF_8);

System.out.println("Document parsed");

# Write Data

### Single record, single bin
The following code will write the entire document to a single bin.

In [ ]:
System.out.println("Writing record...\n");

Key key = new Key("demo", "space-doc", "companies");

JsonNode spaceNode = JsonConverters.convertStringToJsonNode(companiesString);

docClient.put(writePolicy, key, "company_bin", spaceNode);

Record record = client.get(null, key);
System.out.format("Companies: %2.100s\n\n", record.bins.get("company_bin"));

System.out.println("Writing record complete");

# Read Data

### Read entire document 
Return the entry for the company Pixxel.

In [ ]:
System.out.println("Getting company data...\n");
Key key = new Key("demo", "space-doc", "companies");

Object pixxel = docClient.get(key, "company_bin", "$.companies[*].company[?(@.companyName == 'Pixxel')]");

System.out.format("%s\n\n", pixxel);
System.out.println("Company data returned");

### Read specific keys from the document
Return only specific keys from the Pixxel document.

In [ ]:
System.out.println("Getting company data...\n");
Key key = new Key("demo", "space-doc", "companies");

Object pixxel = docClient.get(key, "company_bin", "$.companies[*].company[?(@.companyName == 'Pixxel')].['companyName','about','email','webSite']");

System.out.format("%s\n\n", pixxel);
System.out.println("Company data returned");

### Read specific keys from the document with a complex filter
Return only specific information about a company with more than 2 employee testimonials and a `paymentType` of free. 

In [ ]:
System.out.println("Getting companies data...\n");
Key key = new Key("demo", "space-doc", "companies");

List companyData = (List) docClient.get(key, "company_bin", "$.companies[*].company[?(@.testimonials.length() > 2 && @.paymentType == 'free')].['companyName','about','email','webSite']");

for(int i = 0; i < companyData.size(); i++){
    System.out.format("%s\n\n", companyData.get(i));
}
System.out.println("Company data returned");

# Update Data

## Create new employee testimonial for CERN

In [ ]:
// Create a new employee testimonial for CERN
Map empTest = new TreeMap<String, String>();

empTest.put("content", "My summer internship at CERN was absolutely incredible. The amount of knowledge I have gained while working with some of the best scientists in the world is sure to propel my career forward. I hope to have the opportunity to return to CERN in the future");
empTest.put("person", "Rebecca");
empTest.put("position", "Summer intern at CERN");

System.out.println("Testimonial created");

### Update the CERN entry with the new employee testimonial

In [ ]:
System.out.println("Adding new testimonial...\n");
Key key = new Key("demo", "space-doc", "companies");

// Add the new testimonial
docClient.append(key, "company_bin", "$.companies[*].company[?(@.companyName=='CERN')].testimonials", empTest);

// Read back the testimonials to see the new entry
List test = (List) docClient.get(key, "company_bin", "$.companies[*].company[?(@.companyName=='CERN')]..testimonials");

List testimonials = (List) test.get(0);
for (int i = 0; i < testimonials.size(); i++){
    System.out.format("%s\n\n", testimonials.get(i));
}

System.out.println("Testimonial added");

# Delete Data

Delete all companies with a payment type **NOT** equal to 'free'

In [ ]:
// Get number of companies in the document
Object companies = docClient.get(key, "company_bin", "$.companies.length()");
System.out.println(companies + " Companies in the list\n");

// Delete all companies with a payment type NOT equal to 'free'
docClient.delete(key, "company_bin", "$.companies[?(@.company.paymentType!='free')]");
System.out.println("Companies deleted\n");

// Get new count of companies in the document
companies = docClient.get(key, "company_bin", "$.companies.length()");
System.out.println(companies + " Companies in the list");

## Cleanup

In [ ]:
// Remove the data
client.truncate(null, "demo", "space-doc", null);
System.out.println("Data removed");

client.close();